<a href="https://colab.research.google.com/github/RAFS20/Finance/blob/main/Black_Scholes_Merton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import yfinance as yf
import numpy as np
from scipy.stats import norm
from datetime import datetime

def black_scholes_merton_price(option_type, S, K, T, r, sigma):
    """Compute the Black-Scholes-Merton price for an option."""
    d1 = (np.log(S / K) + (r + (sigma ** 2) / 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type == "call":
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == "put":
        return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)
    else:
        raise ValueError("Invalid option type")

def get_relevant_strike_price(strike_prices, current_price, option_type, position):
    """Retrieve the appropriate strike price based on the option type and position."""
    if position == 'ATM':
        return min(strike_prices, key=lambda x: abs(x - current_price))

    # Dictionary to define the conditions for each type and position
    conditions = {
        'call': {
            'ITM': lambda x: x < current_price,
            'OTM': lambda x: x > current_price
        },
        'put': {
            'ITM': lambda x: x > current_price,
            'OTM': lambda x: x < current_price
        }
    }

    # Filter strike prices based on conditions
    relevant_strikes = [strike for strike in strike_prices if conditions[option_type][position](strike)]
    return max(relevant_strikes) if (option_type == "call" and position == "ITM") or (option_type == "put" and position == "OTM") else min(relevant_strikes)

def fetch_financial_data(ticker_symbol, start_date, end_date):
    """Fetch option and stock data for the provided ticker."""
    try:
        ticker = yf.Ticker(ticker_symbol)
        strike_prices = ticker.option_chain(ticker.options[0]).calls['strike'].tolist() if ticker.options else []
        stock_data = yf.download(ticker_symbol, start=start_date, end=end_date)
        risk_free_data = yf.download("^IRX", start=start_date, end=end_date)
        return strike_prices, stock_data, risk_free_data
    except Exception as e:
        print(f"Error retrieving data: {e}")
        return [], None, None

def extract_option_parameters(stock_data, risk_free_data, expiration_date_str):
    """Extract necessary parameters for Black-Scholes-Merton formula."""
    days_to_expiration = (datetime.strptime(expiration_date_str, '%Y-%m-%d') - stock_data.index[-1].to_pydatetime()).days
    T = days_to_expiration / 252
    S = stock_data["Adj Close"].iloc[-1]
    r = risk_free_data["Adj Close"].iloc[-1] / 100
    log_returns = np.log(stock_data['Adj Close'] / stock_data['Adj Close'].shift(1))
    sigma = log_returns.std() * np.sqrt(252)
    return S, r, sigma, T

def main():
    """Main execution."""
    ticker_symbol = "AAPL"
    start_date, end_date, expiration_date_str = "2023-01-01", "2023-12-31", "2023-12-31"
    option_type = "call"

    strike_prices, stock_data, risk_free_data = fetch_financial_data(ticker_symbol, start_date, end_date)

    # Check for empty dataframes
    if not stock_data.empty and not risk_free_data.empty:
        S, r, sigma, T = extract_option_parameters(stock_data, risk_free_data, expiration_date_str)

        for position in ['ATM', 'ITM', 'OTM']:
            strike = get_relevant_strike_price(strike_prices, S, option_type, position)
            price = black_scholes_merton_price(option_type, S, strike, T, r, sigma)
            print(f"The price of the {option_type} {position} option with a strike price of ${strike:.2f} is: ${price:.2f}")

if __name__ == "__main__":
    main()



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
The price of the call ATM option with a strike price of $175.00 is: $11.06
The price of the call ITM option with a strike price of $172.50 is: $12.42
The price of the call OTM option with a strike price of $175.00 is: $11.06
